# Michaelis-Menten Animated Visualization and Modeling Project Playground

## What is the Michaelis-Menten model?

The model accounts for enzymatic dynamics (how enzymes affect the reaction rate and how the rates themselves depend on the concentration of the enzyme and substrate)

In [136]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [137]:
# Dash sample code for later

app = Dash(__name__)


app.layout = html.Div(
    [
        html.H4("Animated GDP and population over decades"),
        html.P("Select an animation:"),
        dcc.RadioItems(
            id="selection",
            options=["GDP - Scatter", "Population - Bar"],
            value="GDP - Scatter",
        ),
        dcc.Loading(dcc.Graph(id="graph"), type="cube"),
    ]
)


@app.callback(Output("graph", "figure"), Input("selection", "value"))
def display_animated_graph(selection):
    df = px.data.gapminder()  # replace with your own data source
    animations = {
        "GDP - Scatter": px.scatter(
            df,
            x="gdpPercap",
            y="lifeExp",
            animation_frame="year",
            animation_group="country",
            size="pop",
            color="continent",
            hover_name="country",
            log_x=True,
            size_max=55,
            range_x=[100, 100000],
            range_y=[25, 90],
        ),
        "Population - Bar": px.bar(
            df,
            x="continent",
            y="pop",
            color="continent",
            animation_frame="year",
            animation_group="country",
            range_y=[0, 4000000000],
        ),
    }
    return animations[selection]


app.run_server(debug=True)

In [138]:
data = pd.read_csv("vmkmki_mmdata.csv")
data = data.rename(columns = {'S' : 's', 'I' : 'i'})

## Note on the meaning of the data and columns
- **s** represents the concentration of the substrate - **unit**: M
- **i** represents the concentration of the inhibitor - **unit**: M
- **v** represents the reaction rate - **unit**: M/(L * s)

In [139]:
def no_inhibition(vmax, s, m_const):
    return (vmax * s) / (m_const + s)

In [140]:
def competitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = (1 + i / i_const)
    return (vmax * s) / (alpha * m_const + s)

In [141]:
def noncompetitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = 1 + i / i_const
    return ((vmax / alpha) * s) / (s + m_const)

In [142]:
def uncompetitive_inhibition(vmax, s, m_const, i, i_const):
    alpha = 1 + i / i_const
    return ((vmax / alpha) * s) / ((m_const / alpha) + s)

## Lineweaver Burk Plots

With the data only providing the terms S, V_max, and I and not the constants of K_m and K_i we need to calculate them separately using our experimental data.

Note: K_m can be used to solve for K_i depending on the type of the inhibition

In [143]:
def km_solve(df):
    v = 1 / df['v']
    v = v.replace({np.inf: 0})
    s = 1 / df['s']
    s = s.replace({np.inf: 0})

    linmod = sm.OLS(v, s).fit()
    return linmod.params.iloc[0] * df["v"]

In [144]:
m_const = km_solve(data)

In [145]:
i_const = m_const * (data['i'] / (m_const - 1))

In [146]:
data = data.eval(''' 
k_m = @m_const
k_i = @i_const
''')

In [147]:
obs = data[data['s'] > 0]

In [148]:
obs

,s,i,v,k_m,k_i
0,200.0,0.00,18.1,44.457045,0.000000
1,200.0,0.00,18.8,46.176379,0.000000
2,200.0,6.25,17.7,43.474569,6.397147
3,200.0,6.25,18.1,44.457045,6.393820
4,200.0,12.50,16.4,40.281522,12.818216
5,200.0,12.50,17.6,43.228950,12.796005
6,200.0,25.00,13.7,33.649808,25.765701
7,200.0,25.00,15.5,38.070951,25.674382
8,200.0,50.00,11.0,27.018094,51.921740
9,200.0,50.00,12.7,31.193618,51.655979
